This is my first time using Jupyter Lab *and* Python 3, so we'll see how this goes.

In [1]:
from astropy.io import fits
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline

### Read in the data

In [2]:
# This is the file that Song said to start with.
data_file_1 = '/Users/greg/Desktop/redwoodforest/mock_catologue/data/s16a_massive_logmMax_11.45_z_0.25_0.47_mhalo.fits'
hdu1 = fits.open(data_file_1)

So here I'm going to list the useful bits of this data set:

`ra`: Right Ascension

`dec`: Declination
(*These will be useful when I make the galaxy map plot*)

`z_best`: Best redshift value

`logm_max`: I think this is the mass I want for the radius. Or `logmh_host`?


In [3]:
data_table = Table(hdu1[1].data) # For visualization

Now what I want to do is take the galaxy halo masses and assign them a radius. To do this, I will use  `Colossus` (Diemer 2017).

In [5]:
from colossus.cosmology import cosmology
cosmology.setCosmology('planck15')
from colossus.halo import mass_so